# Explorando o tokenizer

In [ ]:
# Vendo se temos uma GPU:
!nvidia-smi

Thu Aug 29 14:08:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import transformers

# pedra, Pedra

# pipeline serve para agrupar todas as funções necessárias da tarefa
# para processar a entrada e gerar a saída
gerbo = transformers.pipeline(
    task="text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device="cuda" # cpu, cuda
)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [ ]:
gerbo

In [ ]:
# modelo escolhinho: 22 blocos de processamento
# faz normalização
# suporta até 2048 tokens de input
# embedding de 32000 tokens: ele conhece 32k palavras
# tarefa dele: recebe um input (texto que será codificado em tokens com o tokenizer),
# e retorna a probabilidade de cada uma das 32k palavras de ser a próxima

gerbo.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Line

In [ ]:
def analisa_e_processa_prompt(modelo, prompt):
  print("="*80)

  # fazendo o processo normal --- codificando
  tokens = modelo.tokenizer.encode(prompt, return_tensors="pt")

  print(f"- prompt original: {prompt}")
  print(f"- token ids:{tokens}")
  print(f"- # de tokens:{len(tokens)}")

  # fazendo o processo reverso --- decodificando
  decoded = modelo.tokenizer.convert_ids_to_tokens(tokens[0])
  print(f"- tokens decodificados: {decoded}")

  first_tokens = modelo.tokenizer.convert_ids_to_tokens(range(5))
  print(f"- primeiros tokens: {first_tokens}")

  # Taquei o prompt pro LLM. Temperatura controla o quao "criativo" o modelo será.
  # É preciso setar do_sample para controlar a temperatura
  output = modelo(prompt, do_sample=True, temperature=0.7)
  print(f"- output do modelo: {output}")
  print(f"- texto gerado efetivamente: {output[0]['generated_text']}")

  print("="*80)
  return

analisa_e_processa_prompt(gerbo, "Once upon a time")

- prompt original: Once upon a time
- token ids:tensor([[   1, 9038, 2501,  263,  931]])
- # de tokens:1
- tokens decodificados: ['<s>', '▁Once', '▁upon', '▁a', '▁time']
- primeiros tokens: ['<unk>', '<s>', '</s>', '<0x00>', '<0x01>']
- output do modelo: [{'generated_text': 'Once upon a time, in a faraway land, there was a young girl named Lily. Lily was a kind-hearted and loving person who always tried to help others. She lived in a small village, where everyone knew each other. Lily had a big heart and loved to go on adventures with her friends. Lily loved to explore the countryside, she loved the smell of the fresh air, and the sound of the birds singing. She loved to ride her bike, and she would often walk to the river for a swim. One day, Lily\'s family took her to the fair. Lily\'s parents had arranged for her to visit the fair while they were out of town. Lily was excited to see all the fun things at the fair, but she was a bit scared at the same time. She remembered a story her